#Configurations

In [1]:
# INPUT FILES
hapticFeatureDir='../1_haptic/protos/haptic_features.pkl.rand17'
imageFeatureDir='../2_image/2_getImageFeatures/minc_features.pkl'
# some original images and partition files
originalImageDir='../0_get_data/data/images_bak/'
indexDir='../0_get_data/data/index_rand17/'

#OUTPUT FILES
imageReformattedFeatureDir="visFs.pkl"

# paths for the old results
staticFeaturePath='../../penn/data_extract_dynamic_static_features/static.pkl'
dynamicFeaturePath='../../penn/data_extract_dynamic_static_features/dynamic.pkl'

# my haptic

In [2]:
#load the haptic features
import pickle
import numpy as np
import copy

hapFs=pickle.load(open(hapticFeatureDir,'r'))

In [3]:
def to2D(array):
    return np.reshape(array, (array.shape[0], -1))

# convert haptic feature: feature[iadj(1-24)]['train'/'test']['feature'/'label']
#eliminate the iadj==7
hapFs=hapFs[0:6]+hapFs[7:]
for i in range(24):
    for phase in ['train', 'test']:
        hapFs[i][phase]['feature']=to2D(hapFs[i][phase]['feature']);

# debug, hack that omit the "slippery" adj
#hapFs[13]=None

In [4]:
from sklearn import svm, preprocessing
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import numpy as np
import copy

def trainSVM(trainFs, trainLabel, testFs, testLabel, useL2norm, usePowerNormalization, useScaler):
    trainFs=copy.deepcopy(trainFs)
    testFs=copy.deepcopy(testFs)
    # some normalization issues
    if usePowerNormalization:
        for fs in [trainFs, testFs]:
            fs=np.sqrt(np.absolute(fs)) * np.sign(fs)
    if useL2norm:
        for fs in [trainFs, testFs]:
            for ii in range(fs.shape[0]):
                fs[ii,:]=fs[ii,:]/np.sqrt(np.dot(fs[ii,:], fs[ii,:]))
    
    if useScaler:
        dynamic_scaler = preprocessing.StandardScaler().fit(trainFs)
        trainFs = dynamic_scaler.transform(trainFs)
        testFs  = dynamic_scaler.transform(testFs)
    
    clf = svm.SVC(C=1, kernel='linear', cache_size=3000)
    
    try:
        clf.fit(trainFs, trainLabel)
    except:
        if all(trainLabel) or all(1-np.array(trainLabel)):
            print "single label observed"
        else:
            print "unknown training error"
        return {'acc':None, 'f1':None, 'auc':None}

    pre=clf.predict(testFs)
    preScore=clf.decision_function(testFs)
    
    acc=accuracy_score(testLabel, pre)
    try: 
        f1=f1_score(testLabel, pre)
    except:
        f1=None
        
    try: 
        auc=roc_auc_score(testLabel, preScore)
    except:
        auc=None
        
    return {'acc':acc, 'f1':f1, 'auc':auc}

In [5]:
adjectives=['absorbent','bumpy','compressible','cool','crinkly','fuzzy','hairy','hard','metallic','nice','porous',
            'rough','scratchy','slippery','smooth','soft',
            'solid','springy','squishy','sticky','textured','thick','thin','unpleasant']

def testHapticFs(hapFs, useL2norm, usePowerNormalization, useScaler):
    ans={'acc':[], 'f1':[], 'auc':[]}
    for i in range(24):
        if hapFs[i]==None:
            print "skipping iadj==", i+1
            for key in ans.keys():
                ans[key] += [None]
            continue
        cur=  trainSVM(hapFs[i]['train']['feature'], hapFs[i]['train']['label'],
                       hapFs[i]['test']['feature'],  hapFs[i]['test' ]['label'], 
                       useL2norm, usePowerNormalization, useScaler)
        for key in ans.keys():
            ans[key] += [cur[key]]
        print "iadj=", i+1," ", adjectives[i], " res=", cur
    return ans

def meaningfulMean(a):
    count=0;
    s=0;
    for item in a:
        if item!=None:
            s += item
            count += 1
    return s/count

# My imaging 

In [6]:
# load the imaging dataset
convResp=pickle.load(open(imageFeatureDir,'r'))
visFs=convResp[2]['inception_5a/output']

In [7]:
# utility functions for the image
def mapImagePathToId():
    files=!ls  {originalImageDir}
    idx=0
    name2id={}
    for f in files:
        if len(f)>4 and (not f.startswith('.')) and f.endswith('.jpg'):
            name2id[f]=idx
            idx += 1
    return name2id
name2id=mapImagePathToId()

def loadLabels(name2id):
    dir=indexDir
    # structure of ans
    # ans[iadj]['train'/'test'/'validate']['labels'/'image_paths']
    ans=[None]*24
    for i in range(24):
        ans[i]={}
        for aset in ['test', 'train', 'validate']:
            ans[i][aset]={}
            if i<6:
                iadj=i+1
            else:
                iadj=i+2
            
            fname=dir+str(iadj)+aset+".txt"
            lines=open(fname,"r").readlines()
            lines=[lines[j] for j in range(len(lines)) if (j%100)<8]
            lines=map(lambda line: line.split(' '), lines)
            
            # get image names and labels
            labels=map(lambda tpl: int(tpl[1].strip()), lines);
            # get image path -> pure file name -> id
            image_paths=map(lambda tpl: name2id[tpl[0].split('/')[-1]], lines);
            
            ans[i][aset]['labels']=labels
            ans[i][aset]['image_ids']=image_paths
    return ans
datasets=loadLabels(name2id)

def avePooling(fs):
    if len(fs.shape)>=4:
        fs=np.mean(fs, 3);
    if len(fs.shape)>=3:
        fs=np.mean(fs, 2);
    fs=fs.reshape((fs.shape[0], np.prod(fs.shape[1:])))
    return fs

visFs=avePooling(visFs);

# convert the vision features
def mergeDatasetAndFeature(visFs, datasets):
    visFs=copy.deepcopy(visFs)
    vis=[None]*24
    for iadj in range(24):
        visi={'train':{}, 'test':{}}
        visi['train']['label']=datasets[iadj]['train']['labels']+datasets[iadj]['validate']['labels']
        trainImages=datasets[iadj]['train']['image_ids']+datasets[iadj]['validate']['image_ids']
        visi['train']['feature']=visFs[trainImages,:]

        visi['test']['label']=datasets[iadj]['test']['labels']
        testImages=datasets[iadj]['test']['image_ids']
        visi['test']['feature']=visFs[testImages,:]

        vis[iadj]=visi
    return vis
visFs=mergeDatasetAndFeature(visFs, datasets)
# end of conversion

# debug, hack that omit the "slippery" adj
#visFs[13]=None

In [8]:
pickle.dump(visFs, open(imageReformattedFeatureDir,"w"))

# merge the two features

In [9]:
# combine vision and haptic
# first check for label consistency, to ensure that the objects are aligned
def checkConsec(arr, L):
    assert(len(arr)%L==0)
    for i in range(len(arr)/L):
        data=arr[i*L:(i+1)*L]
        assert(all(data) or all(1-data))
    
def checkObjAlign(a, b, la, lb):
    for i in range(len(a)/la):
        assert(a[i*la]==b[i*lb])
    
def checkAlign(hapFs, inter1, visFs, inter2):
    for i in range(24):
        if visFs[i]==None or hapFs[i]==None:
            print "skipping iadj=", i+1
            continue
        for phase in ['train', 'test']:
            hl=np.array(hapFs[i][phase]['label'])
            vl=np.array(visFs[i][phase]['label'])
            assert(hl.shape[0]/inter1==vl.shape[0]/inter2)
            checkConsec(hl, inter1)
            checkConsec(vl, inter2)
            checkObjAlign(hl, vl, inter1, inter2)
    print "label consistency checking passed!"
checkAlign(visFs, 8, hapFs, 100)

label consistency checking passed!


In [10]:
# try to combine all features that related to a single object together
def mergeInstance(fs, N):
    for phase in ['train', 'test']:
        assert(fs[0][phase]['feature'].shape[0] % N ==0)
    
    ans=copy.deepcopy(fs)
    for i in range(24):
        if fs[i]==None:
            print "skipping iadj==", i+1, " in merging"
            continue
            
        for phase in ['train','test']:
            ans[i][phase]['feature']=ans[i][phase]['feature'].reshape((ans[i][phase]['feature'].shape[0]/N, -1))
            ans[i][phase]['label']=ans[i][phase]['label'][0::N]
    return ans

In [11]:
def analyze(res):
    ans={}
    for key in ['f1', 'acc', 'auc']:
        ans[key]=meaningfulMean(res[key])
        print key, ': ', ans[key], "\t", 
    print ""
    return ans

# experiments

In [12]:
def printGoodForm(arr):
    for it in arr:
        print it, "\t",
    print

In [13]:
hapResSub=testHapticFs(mergeInstance(hapFs, 10), False, False, False)
res=analyze(hapResSub);
printGoodForm(hapResSub['auc'])
# combining 10 instances is a little useful

iadj= 1   absorbent  res= {'acc': 0.90000000000000002, 'f1': 0.76923076923076927, 'auc': 1.0}
iadj= 2   bumpy  res= {'acc': 0.8833333333333333, 'f1': 0.46153846153846151, 'auc': 0.71599999999999997}
iadj= 3   compressible  res= {'acc': 0.88571428571428568, 'f1': 0.88235294117647056, 'auc': 0.97166666666666668}
iadj= 4   cool  res= {'acc': 0.80000000000000004, 'f1': 0.625, 'auc': 0.94000000000000006}
iadj= 5   crinkly  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.81600000000000006}
iadj= 6   fuzzy  res= {'acc': 0.80000000000000004, 'f1': 0.0, 'auc': 0.11399999999999999}
iadj= 7   hairy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.99199999999999999}
iadj= 8   hard  res= {'acc': 0.96666666666666667, 'f1': 0.95238095238095233, 'auc': 1.0}
iadj= 9   metallic  res= {'acc': 1.0, 'f1': 1.0, 'auc': 0.99999999999999989}
single label observed
iadj= 10   nice  res= {'acc': None, 'f1': None, 'auc': None}
iadj= 11   porous  res= {'acc': 0.81666666666666665, 'f1': 0.620689655172413

/home/yang/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
def oneObj(hapFs, iadj, useL2norm, useScaler):
    ans={'acc':[], 'f1':[], 'auc':[]}
    for i in range(24):
        if hapFs[i]==None:
            print "skipping iadj==", i+1
            for key in ans.keys():
                ans[key] += [None]
            continue
        cur=  trainSVM(hapFs[i]['train']['feature'], hapFs[i]['train']['label'],
                       hapFs[i]['test']['feature'],  hapFs[i]['test' ]['label'], 
                       useL2norm, usePowerNormalization, useScaler)
        for key in ans.keys():
            ans[key] += [cur[key]]
        print "iadj=", i+1," ", adjectives[i], " res=", cur
    return ans

In [15]:
visResSub=testHapticFs(mergeInstance(visFs, 8), False, False, False)
res=analyze(visResSub);
printGoodForm(visResSub['auc'])
# conclusion is combining multiple visual perception is good

iadj= 1   absorbent  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.59999999999999998}
iadj= 2   bumpy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 1.0}
iadj= 3   compressible  res= {'acc': 0.7142857142857143, 'f1': 0.5, 'auc': 1.0}
iadj= 4   cool  res= {'acc': 0.66666666666666663, 'f1': 0.5, 'auc': 0.80000000000000004}
iadj= 5   crinkly  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.0}
iadj= 6   fuzzy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 1.0}
iadj= 7   hairy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.59999999999999998}
iadj= 8   hard  res= {'acc': 0.66666666666666663, 'f1': 0.66666666666666663, 'auc': 0.875}
iadj= 9   metallic  res= {'acc': 0.8571428571428571, 'f1': 0.0, 'auc': 0.33333333333333337}
single label observed
iadj= 10   nice  res= {'acc': None, 'f1': None, 'auc': None}
iadj= 11   porous  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.80000000000000004}
iadj= 12   rough  res= {'acc': 1.0, 'f1': 1.0, 'auc': 1.0}
iadj= 

In [16]:
def crossProd(fs1, sameN1, fs2, sameN2):
    ans=[None]*24
    for i in range(24):
        if fs1[i]==None or fs2[i]==None:
            print "skipping iadj==", i+1, " in merging"
            continue

        ans[i]={}
        for phase in ['train', 'test']:
            ans[i][phase]={}
            
            f1=fs1[i][phase]['feature'];
            f2=fs2[i][phase]['feature'];
            n1=f1.shape[0]
            n2=f2.shape[0]
            assert(n1 % sameN1==0)
            assert(n2 % sameN2==0)
            assert(n1/sameN1 == n2/sameN2)

            ans[i][phase]['feature']=np.zeros((n1*sameN2, f1.shape[1]+f2.shape[1]))
            ans[i][phase]['label']=np.zeros((n1*sameN2))
            ido=0
            for ii in range(n1/sameN1):
                for l1 in range(sameN1):
                    for l2 in range(sameN2):
                        id1=ii*sameN1+l1
                        id2=ii*sameN2+l2
                        ans[i][phase]['feature'][ido,:]=np.concatenate((f1[id1, :], f2[id2,:]))
                        assert(fs1[i][phase]['label'][id1]==fs2[i][phase]['label'][id2])
                        ans[i][phase]['label'][ido]=fs1[i][phase]['label'][id1]
                        ido += 1
    return ans  

In [17]:
prod=crossProd(mergeInstance(hapFs, 10), 10, mergeInstance(visFs, 8), 1)
#prod=crossProd(hapFs, 100, mergeInstance(visFs, 8), 1)

In [20]:
allRes=testHapticFs(prod, True, False, False)
res=analyze(allRes);
printGoodForm(allRes['auc'])

iadj= 1   absorbent  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.70800000000000007}
iadj= 2   bumpy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 1.0}
iadj= 3   compressible  res= {'acc': 0.90000000000000002, 'f1': 0.86792452830188693, 'auc': 1.0}
iadj= 4   cool  res= {'acc': 0.66666666666666663, 'f1': 0.5, 'auc': 1.0}
iadj= 5   crinkly  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.0}
iadj= 6   fuzzy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 1.0}
iadj= 7   hairy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.69000000000000006}
iadj= 8   hard  res= {'acc': 0.81666666666666665, 'f1': 0.78431372549019607, 'auc': 1.0}
iadj= 9   metallic  res= {'acc': 0.8571428571428571, 'f1': 0.0, 'auc': 0.68500000000000005}
single label observed
iadj= 10   nice  res= {'acc': None, 'f1': None, 'auc': None}
iadj= 11   porous  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 1.0}
iadj= 12   rough  res= {'acc': 1.0, 'f1': 1.0, 'auc': 1.0}
iadj= 13   scratchy  re

# reproduce old result 

In [22]:
# testing cases for the original features
import pickle
staticFs=pickle.load(open(staticFeaturePath,'r'))
checkAlign(visFs, 8, staticFs, 10)
print "load finished"
staticRes=testHapticFs(mergeInstance(staticFs, 1), False, False, True)
res=analyze(staticRes);
printGoodForm(staticRes['auc'])

skipping iadj= 10
skipping iadj= 14
label consistency checking passed!
load finished
skipping iadj== 10  in merging
skipping iadj== 14  in merging
iadj= 1   absorbent  res= {'acc': 0.91666666666666663, 'f1': 0.66666666666666663, 'auc': 0.94399999999999995}
iadj= 2   bumpy  res= {'acc': 0.90000000000000002, 'f1': 0.57142857142857151, 'auc': 0.98599999999999999}
iadj= 3   compressible  res= {'acc': 0.7142857142857143, 'f1': 0.67741935483870963, 'auc': 0.86333333333333329}
iadj= 4   cool  res= {'acc': 0.76666666666666672, 'f1': 0.41666666666666663, 'auc': 0.626}
iadj= 5   crinkly  res= {'acc': 0.78333333333333333, 'f1': 0.0, 'auc': 0.504}
iadj= 6   fuzzy  res= {'acc': 0.80000000000000004, 'f1': 0.0, 'auc': 0.16800000000000001}
iadj= 7   hairy  res= {'acc': 0.76666666666666672, 'f1': 0.0, 'auc': 0.47799999999999998}
iadj= 8   hard  res= {'acc': 1.0, 'f1': 1.0, 'auc': 1.0}
iadj= 9   metallic  res= {'acc': 0.94285714285714284, 'f1': 0.74999999999999989, 'auc': 0.98499999999999999}
skipping i

In [23]:
# test dynamic feature
dynamicFs=pickle.load(open(dynamicFeaturePath,'r'))
checkAlign(visFs, 8, dynamicFs, 10)
print "load finished"
dynamicRes=testHapticFs(mergeInstance(dynamicFs, 1), False, False, True)
res=analyze(dynamicRes);
printGoodForm(dynamicRes['auc'])

skipping iadj= 10
skipping iadj= 14
label consistency checking passed!
load finished
skipping iadj== 10  in merging
skipping iadj== 14  in merging
iadj= 1   absorbent  res= {'acc': 0.94999999999999996, 'f1': 0.8421052631578948, 'auc': 0.89599999999999991}
iadj= 2   bumpy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.504}
iadj= 3   compressible  res= {'acc': 0.81428571428571428, 'f1': 0.81690140845070425, 'auc': 0.79916666666666669}
iadj= 4   cool  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.53600000000000003}
iadj= 5   crinkly  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.40599999999999997}
iadj= 6   fuzzy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.48399999999999999}
iadj= 7   hairy  res= {'acc': 0.83333333333333337, 'f1': 0.0, 'auc': 0.32800000000000001}
iadj= 8   hard  res= {'acc': 0.8833333333333333, 'f1': 0.85106382978723405, 'auc': 0.99624999999999997}
iadj= 9   metallic  res= {'acc': 0.91428571428571426, 'f1': 0.57142857142857151, 'auc': 0.

In [24]:
# test the combined method
baseProd=crossProd(mergeInstance(staticFs, 1), 10, mergeInstance(dynamicFs, 1), 10)
baseProdRes=testHapticFs(baseProd, False, False, True)
res=analyze(baseProdRes);
printGoodForm(dynamicRes['auc'])

skipping iadj== 10  in merging
skipping iadj== 14  in merging
skipping iadj== 10  in merging
skipping iadj== 14  in merging
skipping iadj== 10  in merging
skipping iadj== 14  in merging
iadj= 1   absorbent  res= {'acc': 0.91666666666666663, 'f1': 0.66666666666666663, 'auc': 0.94076000000000004}
iadj= 2   bumpy  res= {'acc': 0.90000000000000002, 'f1': 0.57142857142857151, 'auc': 0.98599999999999999}
iadj= 3   compressible  res= {'acc': 0.7142857142857143, 'f1': 0.67741935483870963, 'auc': 0.86415833333333325}
iadj= 4   cool  res= {'acc': 0.76666666666666672, 'f1': 0.41666666666666663, 'auc': 0.626}
iadj= 5   crinkly  res= {'acc': 0.79166666666666663, 'f1': 0.0, 'auc': 0.48870000000000002}
iadj= 6   fuzzy  res= {'acc': 0.80000000000000004, 'f1': 0.0, 'auc': 0.16800000000000004}
iadj= 7   hairy  res= {'acc': 0.76666666666666672, 'f1': 0.0, 'auc': 0.43791999999999998}
iadj= 8   hard  res= {'acc': 1.0, 'f1': 1.0, 'auc': 1.0}
iadj= 9   metallic  res= {'acc': 0.93999999999999995, 'f1': 0.7407